# **1. Setup and Imports**

In [4]:
!pip install transformers datasets torch sklearn
!pip install datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, pipeline
from datasets import load_dataset, Dataset
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/trainset/orientation/orientation-tr-train.tsv'
data = pd.read_csv(file_path, delimiter='\t', encoding='utf-8')


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Mounted at /content/drive


# **2. Printing the Data Set**

In [5]:
print(data.head())

        id                           speaker sex  \
0  tr00000  ca2031caa4032c51980160359953d507   M   
1  tr00001  4cee0addb3c69f6866869b180f90d45f   M   
2  tr00002  b3d7f76d74ec268492f8190ca123a6b2   M   
3  tr00003  722efac7138c8197a9d1e97eed3a8b18   M   
4  tr00004  be82a4ade406ec6774a0a2e38f6957e3   M   

                                                text  \
0  Yeni yasama döneminin ülkemiz için, milletimiz...   
1  Sayın Başkan, değerli milletvekilleri; bugün, ...   
2  Sayın Başkanım, öncelikle yüce Meclisin Başkan...   
3  24’üncü Dönem Meclis Başkanlığına seçilmenizde...   
4  24’üncü Yasama Dönemimizin tüm milletvekilleri...   

                                             text_en  label  
0  Mr. President, dear lawmakers, I salute you, a...      1  
1  Mr. President, members of lawmakers, as I spea...      1  
2  Mr. President, I'm here to share with you the ...      1  
3  Mr. President, under the principles determined...      1  
4  Mr. President, dear lawmakers, I ask 

# **3. Tokenization and Dataset Preparation**

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_and_prepare(data, text_column):
    texts = data[text_column].fillna("").tolist()  # Handle missing values
    encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    return {
        'input_ids': encodings['input_ids'].detach().numpy(),
        'attention_mask': encodings['attention_mask'].detach().numpy(),
        'labels': data['label'].to_numpy()
    }

encoded_data_en = tokenize_and_prepare(data, 'text_en')
encoded_data_orig = tokenize_and_prepare(data, 'text')

train_data, test_data = train_test_split(data, test_size=0.1, stratify=data['label'])

def create_dataset(encoded_data, indices):
    return Dataset.from_dict({
        'input_ids': np.array([encoded_data['input_ids'][i] for i in indices]),
        'attention_mask': np.array([encoded_data['attention_mask'][i] for i in indices]),
        'labels': np.array([encoded_data['labels'][i] for i in indices])
    })

train_dataset_en = create_dataset(encoded_data_en, train_data.index)
test_dataset_en = create_dataset(encoded_data_en, test_data.index)
train_dataset_orig = create_dataset(encoded_data_orig, train_data.index)
test_dataset_orig = create_dataset(encoded_data_orig, test_data.index)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

# **4. Model Training and Evaluation Function**

In [7]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir='./logs',
    report_to='none'  # Disable external logging
)

model_en = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
trainer_en = Trainer(
    model=model_en,
    args=training_args,
    train_dataset=train_dataset_en,
    eval_dataset=test_dataset_en,
    compute_metrics=compute_metrics
)
trainer_en.train()

model_orig = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
trainer_orig = Trainer(
    model=model_orig,
    args=training_args,
    train_dataset=train_dataset_orig,
    eval_dataset=test_dataset_orig,
    compute_metrics=compute_metrics
)
trainer_orig.train()



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.686400,0.682947,0.581784,0.735605,0.581784,1.000000
2,0.686500,0.685194,0.581784,0.735605,0.581784,1.000000
3,0.679600,0.680015,0.581784,0.735605,0.581784,1.000000


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.696000,0.680373,0.581784,0.735605,0.581784,1.000000
2,0.687600,0.684784,0.581784,0.735605,0.581784,1.000000
3,0.680000,0.679887,0.581784,0.735605,0.581784,1.000000


TrainOutput(global_step=5448, training_loss=0.6883049235294919, metrics={'train_runtime': 4661.0157, 'train_samples_per_second': 9.348, 'train_steps_per_second': 1.169, 'total_flos': 1.146427490414592e+16, 'train_loss': 0.6883049235294919, 'epoch': 3.0})

# **5. Zero-Shot Inference and Comparative Analysis**

In [8]:
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sample_texts_en = test_data['text_en'].dropna().sample(2).tolist()
sample_texts_orig = test_data['text'].dropna().sample(2).tolist()

predictions_en = zero_shot_classifier(sample_texts_en, candidate_labels=["left", "right"])
predictions_orig = zero_shot_classifier(sample_texts_orig, candidate_labels=["left", "right"])
print("Zero-shot classification results for English text:", predictions_en)
print("Zero-shot classification results for original language text:", predictions_orig)
eval_results_en = trainer_en.evaluate()
eval_results_orig = trainer_orig.evaluate()

print("Evaluation results for English model:", eval_results_en)
print("Evaluation results for Original Language model:", eval_results_orig)



config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


Zero-shot classification results for English text: [{'sequence': "Mr. President, dear lawmakers, I salute you all. On behalf of our party on behalf of the 2016 budget bill, which is being discussed today. <p> The budgets that are set up for institutions, as well as the pencils set by law and legislation, also have traces of how social life's equations form social hierarchy. The lack of fair use of budgets, the lack of institutional democratisation, the lack of judicial regulations that will raise the bar of fair division and lead to radical democratisation, the underlying legislature activities, the eradication of the separate forces, as a whole, shapes a social order against the peoples and in favor of the sovereigns. We bear witness that our people are getting poorer and more and more corrupt by injustice. <p> Because of the wars in the Middle East, millions of people from Syria want to cross all the roads they can through Turkey to Europe. On these difficult migration journeys, hund

Evaluation results for English model: {'eval_loss': 0.680014967918396, 'eval_accuracy': 0.5817843866171004, 'eval_f1': 0.7356051703877791, 'eval_precision': 0.5817843866171004, 'eval_recall': 1.0, 'eval_runtime': 45.3505, 'eval_samples_per_second': 35.589, 'eval_steps_per_second': 4.454, 'epoch': 3.0}
Evaluation results for Original Language model: {'eval_loss': 0.6798866987228394, 'eval_accuracy': 0.5817843866171004, 'eval_f1': 0.7356051703877791, 'eval_precision': 0.5817843866171004, 'eval_recall': 1.0, 'eval_runtime': 45.2948, 'eval_samples_per_second': 35.633, 'eval_steps_per_second': 4.46, 'epoch': 3.0}
